### 人类活动识别

In [ ]:
from model.RFID_Dataset import RFID_Dataset
from utils.DatasetUtils import DatasetUtils


data_utils = DatasetUtils()

from torchvision import transforms
from model.Normalization import *

transform=transforms.Compose([
    RobustNorm(-35.0, 35.0)
])

train_dir = r"data\RFID\dataset\train"
# train_dir=r"./output/improve_v2_robust"
eval_dir = r"data\RFID\dataset\eval"

# 加载数据集
train_dataset = RFID_Dataset(
    train_dir,
    T=32,
    step=1,
    transform=transform,
)

eval_dataset = RFID_Dataset(
    eval_dir,
    T=32,
    step=1,
    transform=transform,
)

print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 模型组网
from torchkeras import summary
from model.ClassifyNet.CNNClassifyNet import CNNClassifyNet as ClassifyNet
from model.ModelWorker.ClassifyModelWorker import ClassifyModelWorker

input_shape=(1,32,12)
model=ClassifyNet(
    input_shape=input_shape,
    num_classes=3
)

model_worker=ClassifyModelWorker(model)

print(f"{input_shape=}")
model_info=summary(model,input_shape=input_shape)

In [3]:
# 模型准备
from torch.utils.data import DataLoader
from torch import nn
from torch import optim

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=4)

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [4]:
# 模型训练

model_worker.train(
    criterion=loss,
    optimizer=optimizer,
    train_loader=train_loader,
    eval_loader=eval_loader,
    epochs=5,
    enable_board=False,
)

Model will be trained on cuda
Epoch [1/5] begin...


Training: 100%|██████████| 103/103 [00:01<00:00, 101.37step/s, acc=1, loss=0.000425]


Train Loss: 0.2274, Train Acc: 0.9144


Evaluating: 100%|██████████| 205/205 [00:00<00:00, 433.42step/s, acc=1, loss=0]      


Eval samples:818, Eval Loss: 0.0076, Eval Acc: 0.9988
Epoch [2/5] begin...


...

Eval samples:818, Eval Loss: 0.0000, Eval Acc: 1.0000
Epoch [5/5] begin...


Training: 100%|██████████| 103/103 [00:00<00:00, 148.93step/s, acc=1, loss=7.55e-5]


Train Loss: 0.0001, Train Acc: 1.0000


Evaluating: 100%|██████████| 205/205 [00:00<00:00, 512.00step/s, acc=1, loss=0]      

Eval samples:818, Eval Loss: 0.0000, Eval Acc: 1.0000


In [ ]:
# 模型评估

model_worker.evaluate(
    eval_loader=eval_loader,
    criterion=loss,
)

In [5]:
# 评价指标

from torchmetrics import MetricCollection, Accuracy, F1Score,ConfusionMatrix

res=model_worker.execute_metric(
    eval_loader, 
    MetricCollection([
        # Accuracy(task="multiclass", num_classes=10),
        F1Score(task="multiclass", num_classes=3),
        # ConfusionMatrix(task="multiclass", num_classes=10)
    ]),
)

print(res)

Executing: 100%|██████████| 205/205 [00:00<00:00, 453.23step/s]


{'MulticlassF1Score': tensor(1., device='cuda:0')}


In [6]:
# 保存模型
model_worker.save('./output/HAR/HAR_robust.pth')

In [ ]:
# 加载模型
model_worker.load('./output/HAR/HAR.pth')

In [ ]:
from torch.utils.data import DataLoader


test_dataset=RFID_Dataset(
    r"./output/improve_v2_robust",
    # r"data\RFID\dataset\all",
    T=32,
    step=1,
    # transform=transform,
)
print(f"测试集的数据个数: {len(test_dataset)}")

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=8)
model_worker.evaluate(
    eval_loader=test_loader,
    criterion=loss,
)

In [ ]:
# 模型预测

used_dataset = eval_dataset
indexs,inputs,labels = data_utils.select_simple(used_dataset,count=4)
preds,acc = model_worker.predict(inputs=inputs,labels=labels)

print(f"{inputs.shape=}")
print(f"{acc=}")
print(f"indexs:{indexs}")
print(f"labels:{labels}")
print(f"preds :{preds}")